In [ ]:
import os
import openai
from openai import OpenAI
import numpy as np
import pandas as pd
import csv

In [ ]:
openai_api_key = "input your openai api key "

# Basic ChatGPT


In [ ]:
CHATMODELS = ["gpt-35-turbo",'gpt-4', 'gpt-3.5-turbo', 'gpt-4-1106-preview']
DELAY = 1
def query_openai(prompt, model, system=None, temperature=0.7, max_tokens=	4096):
    openai.api_key = os.getenv("OPENAI_API_KEY")
    client = OpenAI(api_key = openai_api_key)

    if model in CHATMODELS:
        messages = []
        if system:
            messages.append({"role": "system", "content": system})
        messages.append({"role": "user", "content": prompt})

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens)
        generation = response.choices[0].message.content
    else:
        response = client.chat.completions.create(
                model=model,
                prompt=prompt,
                temperature=temperature,
                max_tokens=max_tokens)
        generation = response.choices[0].text
    return generation

In [ ]:
system_prompt = "You are an intelligent medical language expert."

In [ ]:
initial_prompt = "You will review the dialogue between the patient and the doctor discussing about various bio-medical topics. This includes analyzing retrieved documents and citations within the doctor's response. Your task is to evaluate the quality of the citations based on the following criteria."

In [ ]:
instruction_together = """
Cited result A and cited result B will be provided. your task is to evaluate each of them separately based on the following criteria.
Additionally, you are required to suggest a total score for each model, indicating them as 'Total Score A: [score], Total Score B: [score]'.

Another important point is please evaluate generously and leniently. 
However, good scores should only be awarded for citations that are included in essential sections or sentences highly relevant to the retrieved document, not for indiscriminate citations
It means that not every sentence requires a citation. 
For instance, in casual conversations like chit-chat, it is appropriate not to include citations. 
Incorrectly citing in unnecessary situations is wrong, while not citing is correct. 
For example, in everyday conversations, such as simply expressing gratitude or asking questions, citations should not be made. 
For everyday dialogue, such as expressing thanks or posing questions, citations should be avoided. 
Scores should be reduced for unnecessary citations and increased when they are appropriately omitted.

Additionally, in cases where answers can be given based on general knowledge without using retrieved documents, it's not necessary to cite. In these situations, not citing is the correct approach. Lastly, answers that don't include any citations at all are judged as not requiring citations for any sentences, so scores should be assigned accordingly.
"""

In [ ]:
criteria_re = """<Criteria> 
Criteria :

Unacceptable (1 point): When the overall response correctly cites the document less than 25%, the overall grade is 1. This means that the model's response does not contain any part of the cited document and cannot be answered using that document at all.
And too many unnecessary citations were made in contexts like chitchat.

Poor (2 points): When the overall response correctly cites the document at least 25% but less than 50%, the overall grade is 2. This means that the model's response can be inferred from the cited source, but the connection is unclear due to many insignificant contents. Another document might provide a better explanation.
And many unnecessary citations were made in contexts like chitchat.

Satisfactory (3 points): When the overall response correctly cites the document at least 50% but less than 75%, the overall grade is 3. This means that the model's response can be inferred from the cited source. However, it omits minor or insignificant content that the instruction requires.
And few unnecessary citations were made in contexts like chitchat.

Excellent (4 points): When the overall response correctly cites the document 75% or more, the overall grade is 4. This means that the cited source clearly explains the model's response, more so than other retrieved documents.
And almost no unnecessary citations were made in contexts like chitchat.
<Criteria> 
"""

In [ ]:
scores = ['Unacceptable (1', 'Poor (2', 'Satisfactory (3', 'Excellent (4']
def extract_scores(text, scores):
    # Finding the score in the text
    for score in scores:
        if score in text:
            # Extracting the numeric value from the score
            score_value = int(score.split('(')[1].split()[0])
            return score_value
  # Return None if no score is found

# Example usage
text = "Overall Grade: Unacceptable (1 point)"
scores = ['Unacceptable (1 point)', 'Poor (2 points)', 'Satisfactory (3 points)', 'Excellent (4 points)']

extracted_score = extract_scores(text, scores)
print("Extracted Score:", extracted_score)



In [ ]:
def eval_cite_together(dataset, output_name1, output_name2, min_range = 0, max_range = 100, revision = False, temperature = 0.7):
    # 0.7, 0.3, 1.4, 0.05
    exp_answer_list = []
    exp_score_list = []
    com_answer_list = []
    com_score_list = []

    if revision == True:
        for i in range(min_range, max_range):
            exp_prompt = initial_prompt + '\n' + '<Cited result A>' + dataset[output_name1].values[i] + '<Cited result A>' + '\n\n\n' + '<Cited result B>' + dataset[output_name2].values[i] + '<Cited result B>' +'\n\n\n' + instruction_together + '\n' + criteria_re
            exp_answer_list.append(query_openai(prompt = exp_prompt, model = 'gpt-4-1106-preview', system = system_prompt, temperature = temperature))

    else:            
        for i in range(min_range, max_range):
            exp_prompt = initial_prompt + '\n' + '<Cited result 1>' + dataset[output_name1].values[i] + '<Cited result 1>' + '\n\n\n' + '<Cited result 2>' + dataset[output_name2].values[i] + '<Cited result 2>' +'\n\n\n' + instruction_together + '\n' + criteria_re
            exp_answer_list.append(query_openai(prompt = exp_prompt, model = 'gpt-4-1106-preview', system = system_prompt, temperature = temperature))

    exp_score_list = extract_scores(exp_answer_list, scores)
    return exp_answer_list, exp_score_list

In [ ]:
# input your directory for csv file
# llama_key = pd.read_csv("jaccard_key.csv")

In [ ]:
toge_answer_list_re, toge_score_list_re = eval_cite_together(dataset, 'vanilla_output', 'jaccard_output', revision = True, temperature = 1)

In [ ]:
with open("your file name", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(toge_answer_list_re)